# Datalab 2025-2026

<div style="line-height:1;">

## Analyse over elektricitiets gebruik in de Nederlandse markt.

**Groepsnummer: 5**

**Auteurs: Lucas Hoetink, Thomas Galais, Mehmet Kilinc, Joris Zondervan**

<div style="line-height:1.6;">

## **Kernpuntenrapport**

Dit notebook presenteert doormiddel van data van het **CBS** een analytisch overzicht van de **Nederlandse energiemarkt**. Na het opschonen en voorbewerken van de rauwe datasets zijn er verschillende analyses op uitgevoerd om inzicht te krijgen in hoe het elekriciteits gebruik verschilt per gemeente en welke factoren dit beïnvloeden. 

Wij hebben datasets van het CBS doormiddel van <code style="color:#4EC9B0;">pandas</code> en <code style="color:#9CDCFE;">numpy</code> onderzocht. Ook hebben wij met <code style="color:#CE9178;">seaborn</code> en <code style="color:#CE9178;">matplotlib</code> meerdere visualisaties gemaakt om de data gemakkelijk in te zien.


---

## **Importeer de benodigde libraries**

Hier importeren wij <code style="color:#4EC9B0;">pandas</code> en <code style="color:#9CDCFE;">numpy</code>. Ook installeren wij eerst de api voor  <code style="color:#4EC9B0;">cbsodata</code> (CBS open data), en importeren we de library in het notebook.

In [ ]:
import pandas as pd
import numpy as np
%pip install cbsodata
import cbsodata
import seaborn as sns
import matplotlib.pyplot as plt

## **Stel de plotting stijl in**

Om de uniformiteit te bewaren in het hele document gaan we alle grafieken dezelfde stijl geven.

In [ ]:
plt.style.use('seaborn-v0_8')
sns.set_palette('colorblind')
sns.set_style("whitegrid")

---

# **Sprint 3 - Data mining / cleaning**

<div style="line-height:1.6;">

### **Kernpuntenrapport**

Dit hoofdstuk presenteert een analytisch overzicht van het elektriciteitsverbruik per gemeente. Ook gaat dit hoofdstuk in op de verschillende factoren die het elektriciteitsverbruik per huishouden beïnvloeden.Als laatste gaan wij aan de hand van kenmerken van een buurt voorspellingen doen over het energieverbruik.

**Belangrijkste bevindingen**:

- De dataset bevat onzettend veel informatie over verschillende gemeentes, met kolommen zoals leeftijd, woning, inkomen, en elektriciteits- of gas-verbruik
- Gender en leeftijd lijken statistisch verwaarloosbare invloed te hebben op elektriciteits gebruik
- Steden met meer twee-onder-een-kap woningen gebruiken gemiddeld minder energie en gas
- Steden met meer tussen woningen / rijtjeshuizen gebruiken meer energie en gas

Deze patronen benadrukken dat de soort huizen waar een stad meer van bouwt effect heeft op het verwachte elektriciteitsverbruik, en daar ook een grote factor in speelt.

---

## **Importeer de dataset**

<!-- Zet de regel afstand voor betere leesbaarheid -->
<div style="line-height:1.6;">

**Hoe we dit doen:**

We beginnen met <code style="color:#4EC9B0;">pandas</code> en `.get_data()` van <code style="color:#4EC9B0;">cbsodata</code> om het dataset `84799NED` op te halen en meteen om te zetten in een <code style="color:#4EC9B0;">pandas DataFrame</code>. 
We gaan meteen door met data verkenning om de dataset beter te begrijpen. De eerste verkenning met `.head()` en `.shape` geeft ons een snel overzicht van de datastructuur.

De dataset heeft `17341` rijen en `119` kolommen.
`.head()` laat ons de bovenste 5 rijen zien. hieruit kunnen we opmaken waar de dataset over gaat. Het is een dataset met informatie over verschillende wijken in Nederland. De informatie bevat onderwerpen zoals leeftijd, partnerschap, afkomst, geboorte en sterfte totalen, huishoudens, en meer. Interessante kolommen voor ons zijn bijvoorbeeld buurten, woningen, en informatie over energieverbruik.

In [ ]:
# Lucas & Mehmet
# Dataset importeren. Omdat de dataset bovenaan wat extra rijen heeft slaan we die over.
df = pd.DataFrame(cbsodata.get_data('84799NED'))

# Dataset uitlezen
print('De dataset heeft de volgende vorm:', df.shape)
print('\nDe bovenste 5 rijen zijn:\n')

df.head()

<!-- Zet de regel afstand voor betere leesbaarheid -->
<div style="line-height:1.6;">

## **1. De dataset filteren**
#### **1.1 Filteren**
We beginnen met een gestructureerd overzicht te creëren van de data die voor ons interessant is. Dit doen wij door de dataset te filteren en om te zetten in DataFrames gesorteerd op de soort regio, bijvoorbeeld `Nederland` is geen wijk. Het is nodig om de dataset op te splitsen in de relevante regiosoorten zodat wij daar later terug naar kunnen referen. De niet-relevante informatie hindert de snelheid van berekeningen, de accuratie van de data, en door deze weg te laten maken wij het onderzoek ook overzichtelijker.
De interessante informatie voor ons in deze dataset is:

- De wijken en buurten kolom, die geeft ons informatie over locatie.
- De gemeente kolom geeft ons informatie over mogelijke groeperingen van data per gemeente.
- Het regio_type kolom geeft de mogelijkheid te sorteren op soort regio.
- De elektriciteitsverbruik kolom geeft ons inzicht in de gebruikte elektriciteit.
- Wij hebben ook kolommen toegevoegd over woningen, inwoners, en gasgebruik.

Wij denken dat deze kolommen ons goed inzicht zullen geven over de data. Wij hebben veel kolommen ook toegevoegd die wij nog niet hebben onderzocht, maar die belangrijk zijn voor later onderzoek.

In [ ]:
# Dataframe buurten maken
# Thomas & Lucas

# definieer de verschillende kololommen georganiseerd
cols_buurten = ['WijkenEnBuurten', 'Gemeentenaam_1', 'SoortRegio_2', 'Bevolkingsdichtheid_33', 'Woningvoorraad_34', 'GemiddeldeWOZWaardeVanWoningen_35', 'PercentageEengezinswoning_36', 'PercentageMeergezinswoning_37', 'PercentageBewoond_38', 'PercentageOnbewoond_39', 'GIHandelEnHoreca_94', 'AfstandTotHuisartsenpraktijk_106', 'AfstandTotGroteSupermarkt_107', 'AfstandTotKinderdagverblijf_108', 'AfstandTotSchool_109', 'OppervlakteLand_112', 'OppervlakteWater_113', 'MateVanStedelijkheid_116', 'Omgevingsadressendichtheid_117']
cols_inwoners = ['k_20HuishoudensMetHoogsteInkomen_77', 'HuishoudensMetEenLaagInkomen_78', 'HuishoudensMetKinderen_31','HuishoudensZonderKinderen_30', 'Bevolkingsdichtheid_33', 'OpleidingsniveauLaag_64', 'OpleidingsniveauMiddelbaar_65', 'OpleidingsniveauHoog_66', 'Mannen_6', 'Vrouwen_7','k_0Tot15Jaar_8', 'k_15Tot25Jaar_9', 'k_25Tot45Jaar_10', 'k_45Tot65Jaar_11', 'k_65JaarOfOuder_12', 'GeboorteTotaal_24', 'GeboorteRelatief_25', 'SterfteTotaal_26', 'SterfteRelatief_27',]
cols_elektriciteitsverbruik = ['GemiddeldElektriciteitsverbruikTotaal_47', 'GemiddeldAardgasverbruikTotaal_55', 'BouwjaarVoor2000_45', 'BouwjaarVanaf2000_46','PercentageWoningenMetStadsverwarming_63',]
cols_woningen = ['GemiddeldeHuishoudensgrootte_32', 'Appartement_48', 'Tussenwoning_49', 'Hoekwoning_50','TweeOnderEenKapWoning_51', 'VrijstaandeWoning_52', 'BedrijfsvestigingenTotaal_91',]

# creeer een masker voor het nieuwe DataFrame
cols = cols_buurten + cols_inwoners + cols_elektriciteitsverbruik + cols_woningen

# creeer het nieuwe DataFrame
df_raw = df[cols]

##### **De gekozen kolommen**

<!--- Thomas --->

Bij sommige kolommen zou je je misschien afvragen waarom we deze kolom precies bij deze sublist hebben gekozen? Hierin leggen wij uit waarom we specifieke kolommen hebben gekozen, die op het eerste oog misschien niet relevant leken:


Woningvoorraad is relevant bij buurten en niet bij woningen omdat de woningsvoorraad een algemeen beeld schets van de hoeveelheid beschikbare woningen in een buurt en we bij woningen echt ingaan op de specifieke details van een woning.

Percentage bewoond en onbewoond zijn relevant bij buurten omdat als een woning onbewoond is zal het verbruik van de woning hoger zijn dan als is dit niet van toepassing is. Ook zal de staat van de woning beter bekeren etc.

Handel en horeca zijn van toepassing bij buurten omdat dit interresant kan zijn om te zien of deze invloed hebben op het energieverbruik.

Opleidingniveau is handig bij inwoners omdat we dan kunnen zien of opleidingniveau invloed heeft op je energievebruik later.

Mannen en vrouwen hebben we gekozen bij dit onderzoek bij inwoners om te kijken of daar een correlatie in is.

Bouwjaar van huizen hebben we bij elektriciteitsverbruik, omdat ons lijkt dat oudere woningen meer verbruiken dan nieuwere woningen (oudere woningen zijn minder goed geisoleerd, etc).

Wat voor een soort woning het is hebben we bij woningen, omdat we verwachten hoe groter of wat voor soort woning je hebt algemeen gezien invloed zal hebben op je energieverbruik.

Bedrijfvestegingen hebben we bij woningen gedaan omdat bedrijven een soort woningen zijn maar bedrijven ook misschien wel meer/minder gebruiken dan 'normale' woningen. Dit is interresant om te onderzoeken.

<!-- Zet de regel afstand voor betere leesbaarheid -->
<div style="line-height:1.6;">

#### **1.2 Relevante kolommen**

Niet alle kolommen van de dataset zijn interessant voor het onderzoek. Wij kunnen van sommige kolommen nu al zeggen dat deze niet relevant zijn. Daarom gaan wij in de volgende stap de kolommen die wij niet nodig hebben verwijderen. Dit zorgt voor een schonere dataset die overzichtelijker is en waar wij makkelijker inzichten uit kunnen krijgen.
 
##### **Verwijderde kolommen:**
<!-- en waarom -->
We hebben bepaalde irrelevante bronnen verwijderd zoals visserij, inkomsten gerelateerd, sterfte, opleidingsniveau, afkomst, en huurwoningen verwijderd omdat deze niet een directe of indirecte invloed hebben op het energieverbruik. Deze hebben individueel een andere invloed op het energieverbruik, maar hebben grootendeels geen correlatie. Dit hebben wij gedaan door een nieuw dataset aan te maken zonder die kolommen.

##### **Behouden kolommen:**
De kolommen die we hebben gehouden hebben we gehouden omdat deze essentieel zijn voor ons onderzoek en onderwerp. En we zonder deze kolommen te weinig informatie zouden bezitten om een goede conclusie te trekken.
De kolommen alsvolgt: Wijken en Buurten, (zonder deze kolommen hebben we minimale informatie tot onze beschikking), woningsoorten, (hoekwoning, twee onder een kap, etc.) vinden wij belangrijk om te houden omdat we met zekerheid kunnen zeggen dat de grootte van uw huis een sterke correlatie heeft met het energieverbruik.
Gemiddeld elektriciteitsverbruik is ook vrij logisch. Met dit gemiddelde kunnen we veel zeggen over het energieverbruik en kunnen we grafieken maken.
Vervolgens hebben we de bouwjaren gehouden van de woningen omdat we denken dat het bouwjaar ook een  correlatie heeft met het gasverbruik, (hoe ouder een huis, hoe slechter geisoleerd, hoe meer gas het huis verbruikt).
En als laatst hebben we de kolommen meergezingswoningen, eensgezinswoningen, bewoond, onbewoond, omdat al deze kolommen essentiële informatie bevatten over huizen waar kinderen in wonen. Dit resulteerd vaak in een hoger energieverbuik. Hierom hebben wij deze kolommen ook gekozen.

Wij hebben ook de verschillende kolommen een nieuwe benaming gegeven om de data overzichtelijker te maken.

In [ ]:
# Lucas
# Renaming columns
buurt_namen = ['WijkenEnBuurten', 'Gemeente', 'Regio_soort', 'Bevolkingsdichtheid', 'Woningvoorraad', 'Gemiddelde_WOZWaarde_Woningen', 'PercentageEengezinswoning', 'PercentageMeergezinswoning', 'PercentageBewoond', 'PercentageOnbewoond', 'GIHandelEnHoreca', 'AfstandTotHuisartsenpraktijk', 'AfstandTotGroteSupermarkt', 'AfstandTotKinderdagverblijf', 'AfstandTotSchool', 'OppervlakteLand', 'OppervlakteWater', 'MateVanStedelijkheid', 'Omgevingsadressendichtheid']
inwon_namen = ['top20_HuishoudensMetHoogsteInkomen', 'HuishoudensMetEenLaagInkomen', 'HuishoudensMetKinderen','HuishoudensZonderKinderen', 'Bevolkingsdichtheid', 'OpleidingsniveauLaag', 'OpleidingsniveauMiddelbaar', 'OpleidingsniveauHoog', 'Mannen', 'Vrouwen','k_0Tot15Jaar', 'k_15Tot25Jaar', 'k_25Tot45Jaar', 'k_45Tot65Jaar', 'k_65JaarOfOuder', 'GeboorteTotaal', 'GeboorteRelatief', 'SterfteTotaal', 'SterfteRelatief',]
wonin_namen = ['Elektriciteitsverbruik', 'Gasverbruik', 'BouwjaarVoor2000', 'BouwjaarVanaf2000','PercentageWoningenMetStadsverwarming',]
elekt_namen = ['GemiddeldeHuishoudensgrootte', 'Appartement', 'Tussenwoning', 'Hoekwoning', 'TweeOnderEenKap', 'Vrijstaand', 'BedrijfsvestigingenTotaal',]

buurt_cols = buurt_namen + inwon_namen + wonin_namen + elekt_namen

df_raw.columns = buurt_cols

df_raw = df_raw.map(lambda x: x.strip() if isinstance(x, str) else x)

land = df_raw[df_raw['Regio_soort'] == 'Land']
gemeenten = df_raw[df_raw['Regio_soort'] == 'Gemeente']
wijken = df_raw[df_raw['Regio_soort'] == 'Wijk']
buurten_raw = df_raw[df_raw['Regio_soort'] == 'Buurt']

<div style="line-height:1.6;">

#### **1.3 Onderbouwing: kolommen met te veel missende waarden**

Binnen onze dataset bevatten sommige kolommen een hoog percentage missende waarden. Kolommen met te veel ontbrekende data kunnen de betrouwbaarheid van analyses verminderen en leiden tot vertekende conclusies. Daarom hebben wij besloten om kolommen met meer dan 30% missende waarden te verwijderen. Deze drempel is gekozen omdat een groter aandeel ontbrekende waarden het lastig maakt om statistische analyses betrouwbaar uit te voeren, terwijl imputatie de resultaten te veel zou beïnvloeden.

In onze dataset zijn de kolommen PercentageWoningenMetStadsverwarming, Appartement, Tussenwoning, en Hoekwoning boven deze drempel uitgekomen en zouden worden verwijderd. Dit wordt gedaan met behulp van de `drop` methode van <code style="color:#4EC9B0;">pandas</code>, waardoor we een schonere en betrouwbaardere dataset krijgen voor verdere analyse.

We behouden enkel wel de Appartement, Tussenwoning, en Hoekwoning kolommen omdat ze nodig zijn voor het onderzoek, en de missende waardes niet uitmaken als wij de data set groeperen op gemeente.

In [ ]:
# Joris & Lucas
# Verwijderen van kolommen met te veel missende waarden
missing_percent = buurten_raw.isna().mean() * 100
missing_percent_sorted = missing_percent.sort_values(ascending=False)

# Opstellen van een plot voor visualisering
plt.figure(figsize=(14,6))
bars = plt.bar(missing_percent_sorted.index, missing_percent_sorted.values, color='mediumslateblue')
plt.xticks(rotation=90)
plt.ylabel("Percentage missende waarden (%)")
plt.xlabel("Kolomnaam")
plt.title("Percentage missende waarden per kolom")
plt.axhline(y=30, color='red', linestyle='--', label='Drempel 30%')
plt.legend()
plt.tight_layout()
plt.show()
threshold = 30

# Behouden van de kolommen appartement, tussenwoning, en hoekwoning
cols_to_drop = missing_percent[missing_percent > threshold].index.tolist()
to_keep = [ 'Appartement', 'Tussenwoning', 'Hoekwoning', 'TweeOnderEenKap']
cols_to_drop = [item for item in cols_to_drop if item not in to_keep]

print("Kolommen die verwijderd worden vanwege te veel missende waarden (>30%):\n", cols_to_drop)

buurten = buurten_raw.drop(columns=cols_to_drop)

# We vullen de lege huizen in met 0
buurten[to_keep] = buurten[to_keep].fillna(0)

print("\nNieuwe datasetvorm na verwijderen kolommen:", buurten.shape)
print("\nOverzicht van missende waarden na verwijderen:")
print(buurten.isna().sum().sort_values(ascending=False))

plt.figure(figsize=(14,6))
bars = plt.bar(missing_percent_sorted.index, missing_percent_sorted.values, color='mediumslateblue')
plt.xticks(rotation=90)
plt.ylabel("Percentage missende waarden (%)")
plt.xlabel("Kolomnaam")
plt.title("Percentage missende waarden per kolom")
plt.axhline(y=30, color='red', linestyle='--', label='Drempel 30%')
plt.legend()
plt.tight_layout()
plt.show()---

<!-- Zet de regel afstand voor betere leesbaarheid -->
<div style="line-height:1.6;">

## **2. Inspecteren**

#### **2.1 Dataset inspecteren**

Nu dat wij de dataset hebben gefiltered kunnen we kijken naar de interessante informatie. We beginnen met een gestructueerd overzicht via de `.info()` methode. Dit geeft ons het aantal rijen, kolommen, de hoeveelheid bestaande waarden, en de kolom namen. Dit is essentieel voor het begrijpen en schoonmaken van onze data.

In [ ]:
# Lucas
# Aantal rijen en kolommen dataframe
buurten.info()

<!-- Zet de regel afstand voor betere leesbaarheid -->
<div style="line-height:1.6;">

<!-- Mehmet -->

#### **2.2 Omschrijving van de kolommen**

Onze dataset heeft `17341` rijen en `49` kolommen. Sommige interessante kolommen hebben wij hieronder beschreven met informatie over wat de kolom is, wat voor type data er in zit, welke eenheden de data in is, en of er missende waarden zijn.


| kolomnaam | omschrijving | type | eenheden | missende waarden |
|------------|--------------|-------|-----------|------------------|
| Wijken_Buurt | Toont aantal wijken en buurten, meerdere buurten vormen een wijk | object | aantal | 0 |
| Gemeente | Toont aantal Gemeenten | object | aantal | 0 |
| Regio_soort | Toont het soort regio, dus gemeente, wijk of buurt | object | aantal | 0 |
| Bevolkingsdichtheid | Toont het aantal inwoners per km² | float64 | inwoners/km² | 690 |
| Woningvoorraad | Het totale aantal woningen op 1 januari | int64 | aantal | 0 |
| Gemiddelde_WOZWaarde_Woningen | Gemiddelde waarde van de woning | float64 | euro | 1770 |
| PercentageEengezinswoning | Percentage van eengezingswoningen (elke woning die een gehele pand vormt) | float64 | % | 1610 |
| PercentageMeergezinswoning | Percentage van meergezinswoningen (elke woning die samen met andere woonruimten een geheel pand vormt) | float64 | % | 1610 |
| PercentageBewoond | Toont percentage bewoonde woningen (woningen waar op 1 januari minimaal 1 persoon stond ingeschreven in de BRP) | float64 | % | 1610 |
| PercentageOnbewoond | Toont percentage onbewoonde woningen (woningen waar op 1 januari niemand stond ingeschreven in de BRP) | float64 | % | 1610 |
| GIHandelEnHoreca | Toont het aantal horeca en handel vestigingen| float64 | % | 2205 |
| AfstandTotHuisartsenpraktijk | Toont gemiddelde afstand van alle inwoners in een gebied tot de dichtsbijzijnde huisartsenpraktijk | float64 | km | 697 |
| AfstandTotGroteSupermarkt | Toont gemiddelde afstand van alle inwoners in een gebied tot de dichtsbijzijnde grote supermarkt | float64 | km | 697 |
| AfstandTotKinderdagverblijf | Toont gemiddelde afstand van alle inwoners in een gebied tot het dichtsbijzijnde kinderdagverblijf | float64 | km | 697 |
| AfstandTotSchool | Toont gemiddelde afstand van alle inwoners in een gebied tot de dichtsbijzijnde school | float64 | km | 697 |
| OppervlakteLand | Toont de gecombineerde informatie van bodemgebruik en gemeente-, wijk-, en buurtgrenzen | int64 | m² | 0 |
| OppervlakteWater | Toont oppervlakte van water, het omvat zowel binnen- als buitenwater | int64 | m² | 0 |
| MateVanStedelijkheid | Op grond van de omgevingsadressendichtheid is aan iedere buurt, wijk of gemeente een stedelijkheidsklasse toegekend. De volgende klassenindeling is gehanteerd: 1: zeer sterk stedelijk >= 2 500 adressen per km² 2: sterk stedelijk 1 500 - 2 500 adressen per km² 3: matig stedelijk 1 000 - 1 500 adressen per km² 4: weinig stedelijk 500 - 1 000 adressen per km² 5: niet stedelijk < 500 adressen per km | float64 | index | 72 |
| Omgevingsadressendichtheid | Toont gemiddelde aantal adressen per km² binnen een cirkel met een straal van één kilometer op 1 januari. | float64 | adressen/km² | 72 |
| top20_HuishoudensMetHoogsteInkomen | Toont aandeel personen in particuliere huishoudens die behoren tot de landelijke 20% personen met hoogste persoonlijk inkomen | float64 | % | 4508 |
| HuishoudensMetEenLaagInkomen | Toont aandeel personen in particuliere huishoudens die behoren tot de landelijke 40% van personen met het laagste persoonlijke inkomen | float64 | % | 4583 |
| HuishoudensMetKinderen | Toont de huishouden met kinderen | int64 | aantal | 0 |
| HuishoudensZonderKinderen | Toont de huishouden zonder kinderen | int64 | aantal | 0 |
| OpleidingsniveauLaag | Toont het aantal personen tussen de 15-75 jaar waarvan de hoogst behaalde niveau laag onderwijs betreft (mbo1, praktijkonderwijs) | float64 | % | 1086 |
| OpleidingsniveauMiddelbaar | Toont het aantal personen tussen de 15-75 jaar waarvan de hoogst behaalde niveau middelbaar onderwijs betreft (vmbo, havo, en mbo) | float64 | % | 1081 |
| OpleidingsniveauHoog | Toont het aantal personen tussen de 15-75 jaar waarvan de hoogst behaalde niveau hoog onderwijs betreft (hbo, wo) | float64 | % | 1086 |
| Mannen | Toont aantal mannen | int64 | aantal | 0 |
| Vrouwen | Toont aantal vrouwen | int64 | aantal | 0 |
| k_0Tot15Jaar | Toont aantal personen met een leeftijd tussen 0 en 15 jaar | int64 | aantal | 0 |
| k_15Tot25Jaar | Toont aantal personen met een leeftijd tussen 15 en 25 jaar  | int64 | aantal | 0 |
| k_25Tot45Jaar | Toont aantal personen met een leeftijd tussen 25 en 455 jaar  | int64 | aantal | 0 |
| k_45Tot65Jaar | Toont aantal personen met een leeftijd tussen 45 en 65 jaar  | int64 | aantal | 0 |
| k_65JaarOfOuder | Toont aantal personen met een leeftijd boven de 65 jaar | int64 | aantal | 0 |
| GeboorteTotaal | Het aantal levendgeborenen van 1 januari tot en met 31 december | int64 | aantal | 0 |
| GeboorteRelatief | Het aantal levendgeborenen van 1 januari tot en met 31 december, per 1000 inwoners | int64 | per 1000 inwoners | 0 |
| SterfteTotaal | Toont aantal overledenen van 1 januari tot en met 31 december | int64 | aantal | 0 |
| SterfteRelatief | Toont aantal overledenen van 1 januari tot en met 31 december, per 1000 inwoners | int64 | per 1000 inwoners | 0 |
| GemiddeldElektriciteitsverbruikTotaal | Toont het gemiddeld jaarverbruik voor elektriciteit op individuele aansluitingen van particuliere woningen, zoals berekend vanuit de aansluitingenregisters van de energienetbedrijven | float64 | kWh | 817 |
| GemiddeldAardgasverbruikTotaal | Toont het gemiddeld jaarverbruik voor aardgas van particuliere woningen, zoals berekend uit de aansluitingenregisters van de energienetbedrijven | float64 | m³ | 1006 |
| BouwjaarVoor2000 | Het aantal woningen met bouwjaar voor 2000, uitgedrukt in hele procenten van het totaal aantal woningen. Het percentage is vermeld bij 20 woningen of meer per buurt| float64 | % | 1610 |
| BouwjaarVanaf2000 | Het aantal woningen met bouwjaar 2000 of later, uitgedrukt in hele procenten van het totaal aantal woningen. Het percentage is vermeld bij 20 woningen of meer per buurt | float64 | % | 1610 |
| PercentageWoningenMetStadsverwarming | Toont het percentage woningen dat is aangesloten op stadsverwarming | float64 | % | 16102 |
| GemiddeldeHuishoudensgrootte | Dit gemiddelde is berekend als het aantal in particuliere huishoudens levende personen gedeeld door het aantal particuliere huishoudens. | float64 | personen | 361 |
| Appartement | Toont het aantal appartementen | float64 | % | 5336 |
| Tussenwoning | Toont het aantal tussenwoningen | float64 | % | 5805 |
| Hoekwoning | Toont het aantal hoekwoningen | float64 | % | 6010 |
| TweeOnderEenKapWoning | Toont het aantal twee onder een kap woningen | float64 | % | 5042 |
| VrijstaandeWoning | Toont aantal vrijstaande woningen | float64 | % | 3089 |
| BedrijfsvestigingenTotaal | Toont het totale bedrijfsvestigingen | int64 | aantal | 0 |




#### **2.3 Extra omschrijving numerieke kolommen**

Om nog beter inzicht in de data te krijgen en ook om de data in perspectief te zetten hebben we hieronder laten zien wat de maximale, minimale, gemiddelde, en mediane waardes zijn in alle numerieke kolommen. Een grote afwijking van de mediaan en het gemiddelde komt vaak door uitschieters, zoals we kunnen zien in de kolommen: Bevolkingsdichtheid en Woningvoorraad. Dit komt vooral door de de verschillende formaten van gemeentes, buurten, en de concentratie van mensen. Er zijn nou eenmaal minder hele grote steden en veel kleine regio's.


| kolomnaam | maximum | minimum | gemiddelde | mediaan |
|------------|----------|----------|-------------|----------|
| Bevolkingsdichtheid | 52173.00 | 0.00 | 2873.60 | 1344.00 |
| Woningvoorraad | 7,891,786.00 | 0.00 | 1820.38 | 407.00 |
| Gemiddelde_WOZWaarde_Woningen | 2069.00 | 19.00 | 311.11 | 286.00 |
| PercentageEengezinswoning | 100.00 | 0.00 | 77.01 | 87.00 |
| PercentageMeergezinswoning | 100.00 | 0.00 | 22.99 | 13.00 |
| PercentageBewoond | 100.00 | 0.00 | 94.43 | 96.00 |
| PercentageOnbewoond | 100.00 | 0.00 | 5.57 | 4.00 |
| GIHandelEnHoreca | 349,180.00 | 0.00 | 91.97 | 20.00 |
| AfstandTotHuisartsenpraktijk | 11.30 | 0.00 | 1.69 | 1.20 |
| AfstandTotGroteSupermarkt | 11.20 | 0.00 | 1.54 | 1.00 |
| BouwjaarVoor2000 | 100.00 | 0.00 | 82.80 | 89.00 |
| BouwjaarVanaf2000 | 100.00 | 0.00 | 17.20 | 11.00 |
| PercentageWoningenMetStadsverwarming | 100.00 | 3.00 | 40.42 | 23.60 |
| GemiddeldeHuishoudensgrootte | 9.00 | 1.00 | 2.30 | 2.30 |
| Appartement | 5750.00 | 770.00 | 2218.09 | 2040.00 |
| Tussenwoning | 6280.00 | 790.00 | 2808.92 | 2800.00 |
| Hoekwoning | 6300.00 | 1170.00 | 2968.11 | 2950.00 |
| TweeOnderEenKapWoning | 7130.00 | 1070.00 | 3384.45 | 3340.00 |
| VrijstaandeWoning | 7140.00 | 1230.00 | 4124.62 | 4120.00 |
| BedrijfsvestigingenTotaal | 2,033,385.00 | 0.00 | 468.92 | 110.00 |




<!-- Zet de regel afstand voor betere leesbaarheid -->
<div style="line-height:1.6;">

#### **2.4 Groeperen kolommen**

Om de dataset beter de begrijpen kunnen wij sommige kolommen, zoals *"Woningtype"* groeperen. Dit helpt ons door meer perspectief te kunnnen geven in de data door middel van gegroepeerde statestieken en door data analyse later makkelijker en duidelijker te maken.

Wij maken hier gebruik van de dataframe `gemeenten` en doormiddel van <code style="color:#4EC9B0;">pandas</code> kunnen wij groeperen op de verschillende woning types en het elektriciteits- en gasvebruik..
Daarna kunnen wij door middel van <code style="color:#CE9178;">seaborn</code> en <code style="color:#CE9178;">matplotlib</code> wat visuele overzichten creëeren om meer inzicht te creëeren in de data.

In [ ]:
# Lucas

### Dataframe opzetten voor de huizen
kolommen = ['Elektriciteitsverbruik','Gasverbruik','Appartement', 'Tussenwoning', 'Hoekwoning', 'TweeOnderEenKap', 'Vrijstaand']
woning_types = ['Appartement', 'Tussenwoning', 'Hoekwoning', 'TweeOnderEenKap', 'Vrijstaand']

woningen_gemeenten = gemeenten[kolommen].copy()

woningen_gemeenten['Woningentotaal'] = woningen_gemeenten[woning_types].sum(axis=1)             # Toevoegen totaal kolom

# Bereken percentages van elk woningtype
wcggs_perc = woningen_gemeenten.copy()
for col in woning_types:
    wcggs_perc[col + '_perc'] = (wcggs_perc[col] / wcggs_perc['Woningentotaal'] * 100)

wcggs = wcggs_perc.sort_values('Elektriciteitsverbruik', ascending=False)                      # Gesorteerde dataset

def correlatie_check(st):
    # Gebruik percentage kolommen voor correlatie
    perc_cols = [col + '_perc' for col in woning_types]
    
    # Melt de data in een langer formaat voor plotten
    df_long = wcggs.melt(
        id_vars=st,
        value_vars=perc_cols,
        var_name='woning_type',
        value_name='percentage'
    )

    # zet de grafieken in 1 deel
    g = sns.FacetGrid(df_long, col='woning_type', col_wrap=3, height=4, sharex=False, sharey=True)
    g.map(sns.regplot, 'percentage', st, scatter_kws={'alpha':0.6}, line_kws={'color':'red'})

    # Geef ze de juiste titels
    for ax, col, perc_col in zip(g.axes.flat, woning_types, perc_cols):
        correlation = wcggs[perc_col].corr(wcggs[st])
        ax.set_title(f"{col}\n(r = {correlation:.2f})", fontsize=10)

    g.set_axis_labels("Percentage van woningtype", st)
    g.figure.suptitle("Relatie tussen woningen en " + st, fontsize=14, y=1.05)
    g.tight_layout()
    plt.show()

correlatie_check('Elektriciteitsverbruik')
correlatie_check('Gasverbruik')

Er lijkt geen sterke correlatie te zijn tussen elektriciteitsgebruik in gemeenten en een specifieke woning soort. Wel kunnen we kijken naar het elektriciteitsverbruik relatief tot de woning soorten. Hieruit blijken een paar inzichten:

**Belangrijkste bevindingen:**

- Er is een zwakke negatieve correlatie tussen 'Tussen woningen' (rijtjeshuizen) en het elektriciteitsverbruik. Maar een zwakke positieve voor appartementen. Dit betekent dat gemiddeld appartementen meer energie gebruiken dan rijtjeshuizen.

- Er is bijna een matige negatieve correlatie tussen de Twee onder een kap en 'Tussen woningen' (rijtjeshuizen) en het Gasgebruik. Deze zijn dus gas zuiniger.

- Appartementen hebben bijna een positief sterke correlatie met gasgebruik, dit betekent dat appartementen veel meer gas gebruiken dan de andere huis soorten.

- Vrijstaande woningen gebruiken niet meer elektriciteit, maar wel meer gas.

<!-- Zet de regel afstand voor betere leesbaarheid -->
<div style="line-height:1.6;">

#### **2.5 Basisstatistieken per gemeente**

Om de data verder te verkennen gaan we nu wat meer basisstatistieken opzoeken over de gemeentes. We gaan de totalen, gemiddleden, en medianen berekenen doormiddel van <code style="color:#4EC9B0;">pandas</code>. Deze geven ons inzicht in verschillen tussen gemeenten. Daaruit kunnen wij conclusies of voorspellingen halen.

Wij hebben de volgende ondervindingen gedaan:

- Het percentage mannen en vrouwen is per gemeente relatief goed verdeeld. Tussen de 47 en de 53%
- Er lijkt geen correlatie te zijn tussen het aantal woningen en de verdeling van mannen en vrouwen, of het elektriciteitsgebruik.
- Er lijken grote verschillen te zijn in leeftijds percentages per gemeente, mogelijk kunnen wij hier nog meer informatie uit halen.
- Er lijkt ook een lichte positieve correlatie te zijn tussen inwoners met een leeftijd van 25-45 en verschillende appartementen, hoekwoningen, en tussenwoningen.

Wij hebben de percentage mannen en vrouwen cijfers ook vergeleken met de cijfers op het CBS en die komen tot minimaal 2 decimalen allemaal overeen.

De cijfers van de leeftijds groepen per gemeente zaten een paar kleine veranderingen in (tot max 2%) wegens de leeftijd van onze dataset. De data op het CBS is uit 2025.

In [ ]:
#Mehmet & Lucas

# Basisstatistieken:
# Groeperen op gemeenten
buurten_grouped = gemeenten.set_index('Gemeente') 

# Percentages uitrekenen
buurten_grouped['perc_mannen'] = ((buurten_grouped['Mannen'] / (buurten_grouped['Mannen'] + buurten_grouped['Vrouwen'])) * 100).round(2)
buurten_grouped['perc_vrouwen'] = ((buurten_grouped['Vrouwen'] / (buurten_grouped['Mannen'] + buurten_grouped['Vrouwen'])) * 100).round(2)

buurten_grouped.describe().round(2)

In [ ]:
# Lucas
# Leeftijden
# Bereken het percentage van elke leeftijdsgroep per gemeente
age_cols = ['k_0Tot15Jaar', 'k_15Tot25Jaar', 'k_25Tot45Jaar', 'k_45Tot65Jaar', 'k_65JaarOfOuder']
total_people = buurten_grouped[age_cols].sum(axis=1)
percentages = buurten_grouped[age_cols].div(total_people, axis=0) * 100
percentages.columns = [col + '_perc' for col in age_cols]
print('Percentage van elke leeftijdsgroep per gemeente:')
display(percentages)

In [ ]:
# Lucas
# Percentages hoogste vrouwen
buurten_grouped[['Mannen','Vrouwen','perc_mannen','perc_vrouwen']].sort_values('perc_mannen')

In [ ]:
#Lucas
# Percentages hoogste mannen
buurten_grouped[['Mannen','Vrouwen','perc_mannen','perc_vrouwen']].sort_values('perc_vrouwen')

In [ ]:
# Lucas
# Correlatie tussen precentage mannen en vrouwen en de woning types

perc_cols = ['perc_mannen', 'perc_vrouwen'] + age_cols
corr_cols = woning_types + ['Elektriciteitsverbruik']

# Select de relevante kolommen
corr_df = pd.merge(buurten_grouped[corr_cols + perc_cols].copy(), percentages, on='Gemeente')

# maak de correlatie matrix
correlation_table = corr_df.corr().loc[corr_cols, perc_cols]
print('Correlatie tussen woningtypes en percentage mannen/vrouwen per gemeente:')
display(correlation_table)

---

<!-- Zet de regel afstand voor betere leesbaarheid -->
<div style="line-height:1.6;">

## 3. **Voorlopige conclusie**

Deze analyse heeft nu al een paar patronen bevonden op het gebruik van elektriciteit en gas. Bijvoorbeeld in het bouwen van huizen zijn twee onder een kap woningen het efficients voor elektriciteit én gas gebruik. Gemeenten zouden kunnen focussen op deze woningsoorten vaker te bouwen om de druk op het elektriciteits en gas-net te verlagen.

**Belangrijkste prestaties:**

<!-- Placeholders -->

- Er zijn `17341` observaties verwerkt, waarmee `97,9%` van de oorspronkelijke dataset is behouden.
- We hebben al veel informatie en data kunnen visualiseren en bruikbare informatie kunnen vaststellen.
- Correlaties tussen elektriciteits- en gas-verbruik en woningsoorten zijn gevonden.
- Er zijn 356 gemeenten in kaart gebracht, waarbij wij de percentages van gender en leeftijdsgroepen zijn berekend.
- Van deze gemeenten zijn er ook correlatiecoëfficienten berekend van verschillende statestieken.

---

# **Sprint 4 - Data Exploration**

<div style="line-height:1.6;">

### **Kernpuntenrapport**

Dit hoofdstuk presenteert meerdere visulisaties en correlaties tussen verschillende variabelen en statestieken in de data. Dit hoofdstuk laat dus visueel en doormiddel van cijfers zien welk verhaal de data verteld.

**Belangrijkste bevindingen**:

- 
- 
- 
- 

Deze patronen benadrukken 

---

<div style="line-height:1.6;">

## **4. Basisstatestieken**

#### **4.1 Gemiddelde grootte huishouden in Nederland**
 **We willen hier berekenen hoeveel inwoners er gemiddeld in een huis in Nederland zitten omdat het interessant lijkt voor ons onderzoek, met deze informatie kunnen we namelijk bereknHet gemiddelde aantal bewoners in een huis in Nederland is dus ongeveer 2.3.**



In [ ]:
# Thomas
gemiddelde_per_huis = buurten['GemiddeldeHuishoudensgrootte'].mean()
print('gemiddeldeHuishoudensgrootte', gemiddelde_per_huis)

gemiddelde_per_huis = buurten['GemiddeldeHuishoudensgrootte'].mean().round(1)
sns.histplot


In [ ]:
# Lucas
# Bereken het gemiddelde huishoudensgrootte per gemeente

huishouden = gemeenten.copy()
huishouden = huishouden[['Gemeente', 'GemiddeldeHuishoudensgrootte']]
huishouden = huishouden.set_index('Gemeente') 

# Maak het histogram
ax = sns.histplot(data = huishouden)

# Bereken en toon het gemiddelde
mean_value = huishouden['GemiddeldeHuishoudensgrootte'].mean()

ax.axvline(mean_value, color='red', linestyle='--', linewidth=2, label=f'Gemiddelde: {mean_value:.2f}')

# Voeg labels en legenda toe
ax.set_xlabel('Gemiddelde Huishoudensgrootte')
ax.set_ylabel('Aantal Gemeenten')
ax.set_title('Verdeling van Gemiddelde Huishoudensgrootte per Gemeente')
ax.legend()
plt.grid(False)

plt.tight_layout()
plt.show()

<div style="line-height:1.6;">

#### **4.2 Aantal inwoners per stedelijkheidscategorie**

In [ ]:
# Thomas

#### **4.3 Gemiddelde elektriciteitsverbruik van een huishouden in Nederland**

In [ ]:
# Thomas

---

<div style="line-height:1.6;">

## **5. Correlaties**

#### **5.1 Correlatie tabel**



In [ ]:
# Joris
# Correlatie van numerieke kolommen met Elektriciteitsverbruik
import matplotlib.pyplot as plt

# Selecteer alleen numerieke kolommen
numeric_cols = buurten.select_dtypes(include='number').columns.tolist()

# Bereken correlatie met Elektriciteitsverbruik
correlations = buurten[numeric_cols].corr()['Elektriciteitsverbruik'].drop('Elektriciteitsverbruik')

correlations

<div style="line-height:1.6;">

#### **5.2 Visualisatie sterkst correlerende variablen**


In [ ]:
# Selecteer top 5 positief en top 5 negatief gecorreleerde variabelen
top5 = correlations.sort_values(ascending=False).head(5)
bottom5 = correlations.sort_values(ascending=True).head(5)

# Combineer en sorteer opnieuw zodat de grafiek netjes is
selected = pd.concat([bottom5, top5]).sort_values()

# Draai de volgorde om: hoog links, laag rechts
selected = selected.sort_values(ascending=False)

# Kleur: rood voor negatief, blauw voor positief
colors = ['red' if val < 0 else 'steelblue' for val in selected]


# Plot
plt.figure(figsize=(10, 6))
selected.plot(kind='bar', color=colors)
plt.title('Top 5 hoogste en laagste correlaties met Elektriciteitsverbruik')
plt.xlabel('Correlatiecoëfficiënt')
plt.ylabel('Variabele')
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

<div style="line-height:1.6;">

#### **5.3 Relatie tussen de categoriale variabelen en elektriciteitsverbruik**


In [ ]:
# Mehmet

cat_cols = ['WijkenEnBuurten', 'Gemeente', 'Regio_soort']
buurten_cat = buurten[cat_cols].copy()
buurten_cat

df_gem = buurten.groupby('Gemeente')['Elektriciteitsverbruik'].mean().sort_values(ascending=False)

plt.figure(figsize=(8,4))
sns.barplot(data=df_gem.tail(10))
plt.title('Laagste gemiddelde elektriciteitsverbruik (10 gemeenten)')
plt.ylabel('Gemiddeld verbruik')
plt.xticks(rotation=45)
plt.ylim((2250,2650))
plt.show()

plt.figure(figsize=(8,4))
sns.barplot(data=df_gem.head(10))
plt.title('Hoogste gemiddelde elektriciteitsverbruik (10 gemeenten)')
plt.ylabel('Gemiddeld verbruik')
plt.xticks(rotation=45)
plt.ylim((4100,4350))
plt.show()

sns.boxplot(data=buurten, x='Regio_soort', y='Elektriciteitsverbruik')
plt.title('Elektriciteitsverbruik per Regio-soort')
plt.show()

<div style="line-height:1.6;">

#### **5.4 Visualisatie elektriciteitsgebruik per stedelijkheidsniveau**


In [ ]:
# Joris
# Groepeer per stedelijkheidsniveau
gemiddeld_per_stedelijkheid = buurten.groupby('MateVanStedelijkheid')['Elektriciteitsverbruik'].mean().sort_index()

# Plot de grafiek
plt.figure(figsize=(8, 5))
gemiddeld_per_stedelijkheid.plot(kind='bar', color='steelblue')
plt.title('Gemiddeld Elektriciteitsverbruik per Stedelijkheidsniveau')
plt.ylabel('Gemiddeld Elektriciteitsverbruik')
plt.xlabel('Stedelijkheidsniveau')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

<div style="line-height:1.6;">
De grafiek toont het gemiddeld elektriciteitsverbruik per stedelijkheidsniveau.
Op de x-as staan de stedelijkheidsniveaus (van 1.0 t/m 5.0).
Op de y-as staat het gemiddeld elektriciteitsverbruik.
De balken laten zien dat het elektriciteitsverbruik toeneemt naarmate het stedelijkheidsniveau hoger wordt.
We kunnen dus zie dat in minder stedelijke gebieden het elektriciteitsverbruik gemiddeld hoger is.

Dat zou logisch kunnen zijn, want in landelijke gebieden:
* zijn woningen vaak groter
* hebben meer elektrische apparaten (zoals warmtepompen, boilers, elektrische verwarming)
* en minder gedeelde voorzieningen (bijv. stadsverwarming)

In stedelijke gebieden daarentegen:
* zijn woningen vaak kleiner
* is vaak sprake van centrale verwarming
* en is de bevolkingsdichtheid hoger, dus minder verbruik per huishouden

De visualisatie draagt bij aan het beantwoorden van de hypothese “Hoe kan je het beste energieverbruik voorspellen?”.
Ze laat zien dat stedelijkheidsniveau een relevante voorspellende variabele is.
In een model om energieverbruik te voorspellen, zou je stedelijkheid dus zeker moeten opnemen, naast andere factoren zoals woningtype, inkomen en huishoudgrootte.

---

<div style="line-height:1.6;">

## **6. Verdelingen**

Hieronder tonen wij de verdelingen van de 10 numerieke variabelen die het meeste invloed hebben op het elektriciteitsverbruik.

In [ ]:
# Mehmet

---

<div style="line-height:1.6;">

## **7. Onderbouwde visualisaties**

In dit onderdeel gaan wij wat visualisaties presenteren over verschillende onderwerpen rondom inwoners om beter inzicht te krijgen in het voorspellen van elektriciteits verbruik

#### **7.1 Inwoners per gemeente**

Als eerste gaan we kijken naar inwoners per gemeente. Wij hebben eerder in deze dataset al verschillende statestieken over gemeenten opgezocht. We kunnen deze zelfde dataset hier ook gebruiken. 


In [ ]:
# Lucas
import matplotlib.patches as patches

buurten_vis = gemeenten.set_index('Gemeente')

buurten_vis.index = buurten_vis.index.str.strip()

buurten_vis['Inwoners'] = (buurten_vis['Mannen'] + buurten_vis['Vrouwen']) / 100_000

buurten_vis = buurten_vis.sort_values('Inwoners', ascending=False)

def show_graph(df = buurten_vis, col_ = 'Inwoners', title = f"100.000 inwoners per Gemeente", bins = 50):
    
    data = df
    col = col_

    plt.figure(figsize=(10,6))

    ax = sns.lineplot(x=data.index, y=data[col], data=data)
    
    ax.set_xticks(range(len(data)))

    ax.set_xticklabels([i
                        if ((i % bins == 0 or i == 3 or i == 8 or i == 32) and i != 0) else "" 
                        for i, label in enumerate(ax.get_xticklabels())
                        ])
    
    ax.set_xlabel('Er zijn 355 gemeenten in Nederland')
    ax.set_ylabel('Inwoners')

    ax.hlines(y=5, xmin=2, xmax=6, color='red', linewidth=1)  
    ax.hlines(y=2, xmin=7, xmax=6*3, color='red', linewidth=1)  
    ax.hlines(y=1, xmin=31, xmax=6*3*3, color='red', linewidth=1)

    ax.vlines(x=2, ymax=5, ymin=0, color='red', linewidth=1, linestyles='--')  
    ax.vlines(x=7, ymax=2, ymin=0, color='red', linewidth=1, linestyles='--')  
    ax.vlines(x=31, ymax=1, ymin=0, color='red', linewidth=1, linestyles='--')

    ax.text(8, 5, "500.000 inwoners", color='black', fontsize=10, verticalalignment='center')
    ax.text(6*3+2, 2, "200.000 inwoners", color='black', fontsize=10, verticalalignment='center')
    ax.text(6*3*3+2, 1, "100.000 inwoners", color='black', fontsize=10, verticalalignment='center')

    ax.text(25, 3.2, "In nederland hebben maar 2% (8) van de gemeenten \nmeer dan 200,000 Inwoners. Amsterdam, Rotterdam, \nen Den Haag zijn de enige met meer dan 500.000", color='black', fontsize=10, backgroundcolor = '0.8')
    ax.text(100, 1.8, "9% (32) van de gemeenten hebben meer dan \n100,000 Inwoners.", color='black', fontsize=10, backgroundcolor = '0.8')

    plt.title(title)
    plt.grid(False)
    plt.show()


show_graph()




<div style="line-height:1.6;">

#### **7.2 Inwoners per stedelijkheidscategorie**

Nu gaan we kijken naar stedelijkheids categorieën. Een stedelijkheids categorie geeft aan hoe veel adressen er gemiddeld per vierkante kilometer.
De standaardcategorieën voor stedelijkheid in Nederland, zoals gehanteerd door het CBS, zijn: zeer sterk stedelijk, sterk stedelijk, matig stedelijk, weinig stedelijk en niet stedelijk. Deze indeling wordt gemaakt op basis van de gemiddelde omgevingsadressendichtheid (oad). 
- Zeer sterk stedelijk: Gemiddelde oad van 2500 of meer adressen per km².
- Sterk stedelijk: Gemiddelde oad van 1500 tot 2500 adressen per km².
- Matig stedelijk: (Niet gespecificeerd in bronnen, maar valt tussen 'sterk' en 'weinig').
- Weinig stedelijk: (Niet gespecificeerd in bronnen, maar valt tussen 'matig' en 'niet').
- Niet stedelijk: (Niet gespecificeerd in bronnen, maar is het minst dichtbevolkt). 

In de dataset staan deze als 1.0, 2.0, etc, waar een lager cijfer een hogere stedelijkheid is.

In [ ]:
stedelijkheid_labels = ['Zeer Sterk', 'Sterk', 'Matig', 'Weinig', 'Niet']

def show_stedelijkheid(data = buurten, datax = gemeenten):
    data1 = data.groupby('MateVanStedelijkheid').sum()
    data2 = datax.groupby('MateVanStedelijkheid')['Gemeente'].count()

    print(data2)

    # Bereken totaal aantal inwoners per stedelijkheidsklasse in miljoenen
    data1['Inwoners_miljoen'] = (data1['Mannen'] + data1['Vrouwen']) / 1_000_000
    data1['Inwoners_miljoen'] = data1['Inwoners_miljoen'].round(2)

    print(data1['Inwoners_miljoen'])

    fig, ax1 = plt.subplots(figsize=(12,6))

    # Breedte van de balken en posities
    bar_width = 0.35
    x_pos = np.arange(len(stedelijkheid_labels))

    # Staafdiagram voor aantal inwoners in miljoenen
    bars1 = ax1.bar(x_pos - bar_width/2, data1['Inwoners_miljoen'], bar_width, 
                     color='steelblue', alpha=0.7, label='Aantal inwoners (miljoen)')
    ax1.set_ylabel('Aantal inwoners (miljoen)', color='steelblue', fontweight='bold')
    ax1.set_xlabel('Mate van stedelijkheid')
    ax1.set_xticks(x_pos)
    ax1.set_xticklabels(stedelijkheid_labels)
    ax1.tick_params(axis='y', labelcolor='steelblue')

    # Y-as limiet verhogen
    max_inwoners = data1['Inwoners_miljoen'].max()
    ax1.set_ylim(0, max_inwoners * 1.15)
    ax1.grid(False)

    # Tweede y-as voor aantal gemeenten
    ax2 = ax1.twinx()
    bars2 = ax2.bar(x_pos + bar_width/2, data2.values, bar_width, 
                     color='red', alpha=0.5, label='Aantal gemeenten')
    ax2.set_ylabel('Aantal gemeenten', color='red', fontweight='bold')
    ax2.tick_params(axis='y', labelcolor='red')

    ax1.text(-0.2, 3, "Maar 6% (21) van de gemeenten \nzijn 'Zeer Stedelijk', maar bevatten \n24% (4.14 miljoen) van de inwoners.", color='black', fontsize=10, backgroundcolor = '0.8')
    ax1.annotate('', xy=(0, 2), xytext=(0.4, 2.93), arrowprops={'arrowstyle': '->', 'color': 'steelblue'})

    ax2.text(1.5, 100, "Een groot gedeelte, 36% (128), van de gemeenten \nzijn 'Zwak Stedelijk'. Dit komt doordat Nederland \nveel kleine gemeenten heeft.", color='black', fontsize=10, backgroundcolor = '0.8')
    ax2.annotate('', xy=(3, 85), xytext=(2, 97.5), arrowprops={'arrowstyle': '->', 'color': 'red'})

    plt.title('Aantal inwoners (miljoen) en gemeenten per stedelijkheidsklasse')
    ax2.grid(False)
    plt.show()

show_stedelijkheid(data = buurten)



<div style="line-height:1.6;">

#### **7.3 Drie grafieken die beter inzicht in de data geven**

We gaan nu kijken naar wat meer grafieken om inzicht te krijgen in de data. 


In [ ]:
# Lucas
# Correlatie van numerieke kolommen met Elektriciteitsverbruik
import matplotlib.pyplot as plt
import pandas as pd

# Selecteer alleen numerieke kolommen
numeric_cols = buurten.select_dtypes(include='number').columns.tolist()

# Bereken correlatie met Elektriciteitsverbruik
correlations = buurten[numeric_cols].corr()['Elektriciteitsverbruik'].drop('Elektriciteitsverbruik')

# Sorteer op sterkte van correlatie
correlations_sorted = correlations.apply(lambda x: -x if x < 0 else x).sort_values(ascending=False)

# Plot
plt.figure(figsize=(12,6))
plt.xticks(rotation=90)

def plotting(x, y, xlabel, ylabel, title, type_ = 'line', clr = 'blue'):
    cl = clr

    match type_:
        case 'blue':
            cl = clr
        case 'grey':
            cl = clr

    match type_:
        case 'line':
            ax = sns.lineplot(x=x, y=y, color=cl)
        case 'bar':
            ax = sns.barplot(x=x, y=y, color=cl)

    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)

    plt.title(title)
    plt.grid(False)
    plt.tight_layout()

plotting(correlations_sorted.index, correlations_sorted, 'Numerieke kolommen', 'Correlatie met Elektriciteitsverbruik', 'Correlatie van numerieke kolommen met Elektriciteitsverbruik', type_ = 'bar', clr='grey')
for i in range(0,5):
    plt.bar(i, correlations_sorted.iloc[i], color='blue')
plt.show()


belangrijke_data = buurten[['Gasverbruik', 'GemiddeldeHuishoudensgrootte', 'MateVanStedelijkheid', 'PercentageEengezinswoning', 'Gemiddelde_WOZWaarde_Woningen','PercentageMeergezinswoning', 'Elektriciteitsverbruik', 'Bevolkingsdichtheid']]


plotting(belangrijke_data['GemiddeldeHuishoudensgrootte'], belangrijke_data['Elektriciteitsverbruik'], 'Gemiddelde Huishoudensgrootte', 'Gemiddeld Elektriciteitsverbruik', 'Huishoudensgrootte vs Elektriciteitsverbruik')
ax2 = sns.histplot(data=belangrijke_data['GemiddeldeHuishoudensgrootte'])
ax2.set_xlim((1,5))
plt.xticks(rotation=90)
plt.show()

plotting(belangrijke_data['PercentageEengezinswoning'], belangrijke_data['Elektriciteitsverbruik'], 'Gemiddelde PercentageEengezinswoning', 'Gemiddeld Elektriciteitsverbruik', 'PercentageEengezinswoning vs Elektriciteitsverbruik')
plotting(belangrijke_data['PercentageMeergezinswoning'], belangrijke_data['Elektriciteitsverbruik'], 'Gemiddelde PercentageMeergezinswoning', 'Gemiddeld Elektriciteitsverbruik', 'PercentageEengezinswoning vs Elektriciteitsverbruik')
ax2 = sns.histplot(data=belangrijke_data['PercentageEengezinswoning'])
ax2 = sns.histplot(data=belangrijke_data['PercentageMeergezinswoning'])
plt.xticks(rotation=90)
plt.show()

plotting(belangrijke_data.iloc[:, -1], belangrijke_data['Elektriciteitsverbruik'], 'Gemiddelde Bevolkingsdichtheid', 'Gemiddeld Elektriciteitsverbruik', 'Bevolkingsdichtheid vs Elektriciteitsverbruik')
ax2 = sns.histplot(data=belangrijke_data['Bevolkingsdichtheid'])
plt.xticks(rotation=90)
plt.show()



Om te beginnen willen we kijken naar de correlaties tussen verschillende kolommen en elektriciteitsverbruik. Deze correlatie zal ons vertellen hoe de relatie tussen de kolom en elektriciteitsverbruik is. 

De 5 hoogst correlerende kolommen zijn: 
- GemiddeldAardgasverbruikTotaal: 0.81
- top20_HuishoudensMetHoogsteInkomen: 0.74
- Appartement: 0.7
- GemiddeldeHuishoudensgrootte: 0.67
- MateVanStedelijkheid: 0.64

“Een verblijfsobject (vbo) waarvan de gebruiksfunctie in de BAG tenminste een woonfunctie heeft (eventueel naast nadere gebruiksfuncties), die ligt in een pand zonder andere vbo’s. Hieronder vallen vrijstaande woningen, aaneengeschakelde woningen, hoek- en tussenwoningen, voorzover niet gesplitst of bestaande uit meerdere vbo’s.”

---

<div style="line-height:1.6;">

## **8. Conclusie** 

Dit is onze voorlopige conclusie:

**Opvallende zaken:**

- 
- 
- 

**Voorlopig antwoord op de hoofdvraag:**

xx xxx xxxx

# **Sprint 5**

# **Sprint 6**

### Eindadvies